In [1]:
import sys, os, decimal, json, pathlib, glob, errno
from tqdm import tqdm
import pandas as pd
import seaborn as sns
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt
import mplcyberpunk
plt.style.use("cyberpunk")
plt.rcParams['figure.facecolor'] = '#0d1117'
plt.rcParams['axes.facecolor'] = '#0d1117'
plt.rcParams['savefig.facecolor'] = '#0d1117'

In [2]:
image_names = []
path = pathlib.Path(r'O:\eve_models\training_data\route_y')
for data in glob.glob(f"{path}\*.png"):
    file = data.split('\\')[-1:][0]
    image_names.append(file)

In [3]:
image_data = {}
f = open(f"{path}\\boxes.json")
image_data_raw = json.load(f)
f.close()

for data in image_data_raw:
    image_data[data['image_name']] = [data['bbox'][0:1][0], data['bbox'][1:2][0]]
    
image_x = []
image_y = []
for image_name in tqdm(image_names):
    image_x.append(image_data[image_name][0])
    image_y.append(image_data[image_name][1])

100%|█████████████████████████████████████████████████████████████████████████| 1739/1739 [00:00<00:00, 1739126.05it/s]


In [18]:
min_x = 100
max_x = 100
min_y = 100
max_y = 90

In [19]:
def distance(x, y):
    if x >= y:
        result = (x - y) * -1
    else:
        result = y - x
    return result

In [20]:
a_image_names = image_names.copy()
a_image_x = image_x.copy()
a_image_offset_x = [0] * len(image_x)
a_image_y = image_y.copy()
a_image_offset_y = [0] * len(image_x)
orig_image_names = image_names.copy()

for i, image_name in tqdm(enumerate(image_names)):
    rand_x_ints = list(np.unique(np.random.randint(low=image_x[i] - min_x, high=image_x[i] + max_x, size=10)))
    rand_y_ints = list(np.unique(np.random.randint(low=image_y[i] - min_y, high=image_y[i] + max_y, size=10)))
   
    for j, x in enumerate(rand_x_ints):
        mod_image_name = image_name.replace('.png',f'-{x}-{image_y[i]}.png')
        a_image_names.append(mod_image_name)
        a_image_x.append(x)
        a_image_y.append(image_y[i])
        orig_image_names.append(image_name)
        a_image_offset_x.append(distance(image_x[i], x))
        a_image_offset_y.append(0)
        
    for j, y in enumerate(rand_y_ints):
        mod_image_name = image_name.replace('.png',f'-{image_x[i]}-{y}.png')
        a_image_names.append(mod_image_name)
        a_image_x.append(image_x[i])
        a_image_y.append(y)
        orig_image_names.append(image_name)
        a_image_offset_x.append(0)
        a_image_offset_y.append(distance(image_y[i], y))

1739it [00:00, 24480.18it/s]


In [21]:
len(orig_image_names)

35726

In [22]:
data = { 'orig_image_name':orig_image_names, 'image_names':a_image_names, 'x':a_image_x, 'y':a_image_y, 'x_offset':a_image_offset_x, 'y_offset':a_image_offset_y}
df = pd.DataFrame(columns=['orig_image_name','image_names','x', 'y', 'x_offset', 'y_offset'], data=data)
df

,orig_image_name,image_names,x,y,x_offset,y_offset
0,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,134,343,0,0
1,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,134,330,0,0
2,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,137,296,0,0
3,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,137,255,0,0
4,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,135,206,0,0
...,...,...,...,...,...,...
35721,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-279.png,135,279,0,26
35722,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-287.png,135,287,0,34
35723,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-289.png,135,289,0,36
35724,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-294.png,135,294,0,41


In [23]:
df[df['orig_image_name'] == '001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png']

,orig_image_name,image_names,x,y,x_offset,y_offset
0,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,134,343,0,0
1739,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-42-343.png,42,343,-92,0
1740,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-61-343.png,61,343,-73,0
1741,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-79-343.png,79,343,-55,0
1742,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-109-343.png,109,343,-25,0
1743,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-151-343.png,151,343,17,0
1744,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-152-343.png,152,343,18,0
1745,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-171-343.png,171,343,37,0
1746,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-173-343.png,173,343,39,0
1747,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb-204-343.png,204,343,70,0


In [24]:
#data = { 'image_names':image_names, 'x':image_x, 'y':image_y}
#df = pd.DataFrame(columns=['image_names','x', 'y'], data=data)
#df

In [25]:
#sns.histplot(df[['y']], bins=60)

In [26]:
df['x_bins'] = pd.cut(df['x'], bins=range(0,500,10))
df['y_bins'] = pd.cut(df['y'], bins=range(0,600,10))
df

,orig_image_name,image_names,x,y,x_offset,y_offset,x_bins,y_bins
0,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,134,343,0,0,"(130, 140]","(340, 350]"
1,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,134,330,0,0,"(130, 140]","(320, 330]"
2,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,137,296,0,0,"(130, 140]","(290, 300]"
3,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,137,255,0,0,"(130, 140]","(250, 260]"
4,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,135,206,0,0,"(130, 140]","(200, 210]"
...,...,...,...,...,...,...,...,...
35721,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-279.png,135,279,0,26,"(130, 140]","(270, 280]"
35722,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-287.png,135,287,0,34,"(130, 140]","(280, 290]"
35723,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-289.png,135,289,0,36,"(130, 140]","(280, 290]"
35724,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-294.png,135,294,0,41,"(130, 140]","(290, 300]"


In [29]:
df['bin_id_x'] = 0
df['bin_id_y'] = 0
for i in tqdm(df.index):
    df.loc[i, 'bin_id_x'] = df.loc[i, 'x_bins'].left
    df.loc[i, 'bin_id_y'] = df.loc[i, 'y_bins'].left
df['tick'] = 1
df
#df.groupby('bin_id')['tick'].sum().sort_values(ascending=True)

100%|██████████████████████████████████████████████████████████████████████████| 35726/35726 [00:05<00:00, 7015.58it/s]


,orig_image_name,image_names,x,y,x_offset,y_offset,x_bins,y_bins,bin_id_x,bin_id_y,tick
0,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,134,343,0,0,"(130, 140]","(340, 350]",130,340,1
1,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,134,330,0,0,"(130, 140]","(320, 330]",130,320,1
2,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,137,296,0,0,"(130, 140]","(290, 300]",130,290,1
3,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,137,255,0,0,"(130, 140]","(250, 260]",130,250,1
4,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,135,206,0,0,"(130, 140]","(200, 210]",130,200,1
...,...,...,...,...,...,...,...,...,...,...,...
35721,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-279.png,135,279,0,26,"(130, 140]","(270, 280]",130,270,1
35722,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-287.png,135,287,0,34,"(130, 140]","(280, 290]",130,280,1
35723,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-289.png,135,289,0,36,"(130, 140]","(280, 290]",130,280,1
35724,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-294.png,135,294,0,41,"(130, 140]","(290, 300]",130,290,1


In [30]:
df['bin_x_bin_y'] = df['bin_id_x'].astype(str) + "_" + df['bin_id_y'].astype(str)
df

,orig_image_name,image_names,x,y,x_offset,y_offset,x_bins,y_bins,bin_id_x,bin_id_y,tick,bin_x_bin_y
0,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png,134,343,0,0,"(130, 140]","(340, 350]",130,340,1,130_340
1,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,0014a4f1-d4d4-11ed-8150-2cf05d9fe8eb.png,134,330,0,0,"(130, 140]","(320, 330]",130,320,1,130_320
2,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,001c3640-d343-11ed-9f40-2cf05d9fe8eb.png,137,296,0,0,"(130, 140]","(290, 300]",130,290,1,130_290
3,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,00684465-d4d2-11ed-be24-2cf05d9fe8eb.png,137,255,0,0,"(130, 140]","(250, 260]",130,250,1,130_250
4,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,00789398-d4ce-11ed-8349-2cf05d9fe8eb.png,135,206,0,0,"(130, 140]","(200, 210]",130,200,1,130_200
...,...,...,...,...,...,...,...,...,...,...,...,...
35721,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-279.png,135,279,0,26,"(130, 140]","(270, 280]",130,270,1,130_270
35722,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-287.png,135,287,0,34,"(130, 140]","(280, 290]",130,280,1,130_280
35723,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-289.png,135,289,0,36,"(130, 140]","(280, 290]",130,280,1,130_280
35724,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb.png,ffe54612-d4d2-11ed-a65c-2cf05d9fe8eb-135-294.png,135,294,0,41,"(130, 140]","(290, 300]",130,290,1,130_290


In [31]:
df.groupby(['bin_x_bin_y'])['tick'].sum().sort_values()

bin_x_bin_y
40_260        1
190_480       1
210_470       1
90_260        1
170_260       1
           ... 
130_250     842
130_370     869
130_330     883
130_290     913
130_340    1058
Name: tick, Length: 528, dtype: int64

In [32]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise
        
def copy(source, destination):
    with open(source, 'rb') as file:
        myFile = file.read()
    with open(destination, 'wb') as file:
        file.write(myFile)

In [33]:
black_image = np.array([[[0]*3]*500]*600)
black_image = Image.fromarray(black_image, mode='RGB')

In [37]:
import os, os.path
path = r'O:\eve_models\training_data\route_y_large_vert_class_v4'
unique_bins = df['bin_x_bin_y'].unique()
for bin_id in unique_bins:
    mkdir_p(f"{path}\\{bin_id}")

In [38]:
source = r'O:\eve_models\training_data\route_y'
image_preload = {}
for name in image_names:
    image_preload[name] = Image.open(f"{source}\\{name}")

In [39]:
#source = r'O:\eve_models\training_data\route_y'
#dest = r'O:\eve_models\training_data\route_y_large_vert_class_v4'
#for i in tqdm(df.index):
#    black_image = np.array([[[0]*3]*500]*600)
#    black_image = Image.fromarray(black_image, mode='RGB')
#    #this_source = f"{source}\\{df.loc[i, 'orig_image_name']}"
#    img = image_preload[df.loc[i, 'orig_image_name']]
#    black_image.paste(img, (df.loc[i, 'x_offset'], df.loc[i, 'y_offset']))
#    this_dest = f"{dest}\\{df.loc[i, 'bin_x_bin_y']}\\{df.loc[i, 'image_names']}"
#    black_image.save(this_dest)
    #print(df.loc[i, 'bin_x_bin_y'])
    #break
    #copy(this_source, this_dest)

In [40]:
source = r'O:\eve_models\training_data\route_y'
dest = r'O:\eve_models\training_data\route_y_large_vert_class_v4'

ex_orig_image_name = df['orig_image_name']
ex_image_names = df['image_names']
ex_x_offset = df['x_offset']
ex_y_offset = df['y_offset']
ex_bin_x_bin_y = df['bin_x_bin_y']

black_image = np.array([[[0]*3]*500]*600)
black_image = Image.fromarray(black_image, mode='RGB')

for i, name in tqdm(enumerate(ex_orig_image_name)):
    img = image_preload[ex_orig_image_name[i]]
    black_image_copy = black_image.copy()
    black_image_copy.paste(img, (ex_x_offset[i], ex_y_offset[i]))
    this_dest = f"{dest}\\{ex_bin_x_bin_y[i]}\\{ex_image_names[i]}".replace(".png",".bmp")
    black_image_copy.save(this_dest)


35726it [01:55, 309.85it/s]
